In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### RNN 구현
- 이 코드는 간단한 RNN 모델을 정의한다.
    - `__init__` 함수에서는 필요한 레이어를 정의하고
    - `forward` 함수에서는 이 레이어들을 어떻게 연결할지를 정의한다.

In [3]:
class SimpleRNN(nn.Module): # SimpleRNN 클래스 선언
    def __init__(self, input_size, hidden_size, output_size): 
        super(SimpleRNN, self).__init__() # nn.Module의 초기화 함수 상속
        self.M = hidden_size # hidden state의 크기 지정
        self.D = input_size # 입력 차원의 크기 지정
        self.K = output_size # 출력 차원의 크기 지정
        self.rnn = nn.RNN( # RNN 모듈 생성
            input_size=self.D, # 입력 차원의 크기
            hidden_size=self.M, # hidden state의 크기
            nonlinearity='tanh', # 활성화 함수로 tanh 사용
            batch_first=True # 첫 번째 차원이 batch size임을 명시
        )
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환 정의

    def forward(self, X): # 순전파 함수 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(device) # 초기 hidden state를 0으로 초기화

        out, _ = self.rnn(X, h0) # RNN에 입력을 전닳하고 출력과 hidden state를 받음

        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행
        return out

### RNN 학습

In [4]:
model = SimpleRNN(input_size=2, hidden_size=20, output_size=2).to(device) # 모델 생성
criterion = nn.CrossEntropyLoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

# 더미 데이터 생성
inputs = torch.from_numpy(np.array([[[1, 2], [3, 4], [5, 6]]], dtype=np.float32)).to(device)

for epoch in range(300):
    model.zero_grad() # 기울기 초기화
    outputs = model(inputs) # 모델 출력 계산
    loss = criterion(outputs, torch.tensor([0]).to(device)) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 30 == 0: # 30 에폭마다 손실 출력
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 300, loss.item()))

Epoch [30/300], Loss: 0.2563
Epoch [60/300], Loss: 0.0810
Epoch [90/300], Loss: 0.0424
Epoch [120/300], Loss: 0.0282
Epoch [150/300], Loss: 0.0208
Epoch [180/300], Loss: 0.0163
Epoch [210/300], Loss: 0.0132
Epoch [240/300], Loss: 0.0110
Epoch [270/300], Loss: 0.0093
Epoch [300/300], Loss: 0.0081


### LSTM 구현

In [5]:
class LSTM(nn.Module): # LSTM 클래스 선언
    def __init__(self, input_size, hidden_size, output_size): 
        super(LSTM, self).__init__() # nn.Module의 초기화 함수 상속
        self.D = input_size # 입력 차원의 크기 지정
        self.M = hidden_size # hidden state의 크기 지정
        self.K = output_size # 출력 차원의 크기 지정
        self.lstm = nn.LSTM( # LSTM 모듈 생성
            input_size=self.D, # 입력 차원의 크기
            hidden_size=self.M, # hidden state의 크기
            batch_first=True # 첫 번째 차원이 batch size임을 명시
        )
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환 정의

    def forward(self, X): # 순전파 함수 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(device) # 초기 hidden state를 0으로 초기화
        c0 = torch.zeros(1, X.size(0), self.M).to(device) # 초기 cell state를 0으로 초기화

        out, _ = self.lstm(X, (h0, c0)) # LSTM에 입력을 전달하고 출력과 hidden state를 받음

        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

### LSTM 학습

In [8]:
model = LSTM(input_size = 2, hidden_size=20, output_size=2).to(device) # 모델 생성
criterion = nn.CrossEntropyLoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

for epoch in range(300):
    model.zero_grad() # 기울기 초기화
    outputs = model(inputs) # 모델 출력 계산
    loss = criterion(outputs, torch.tensor([1]).to(device)) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 30 == 0: 
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 300, loss.item()))

Epoch [30/300], Loss: 0.4870
Epoch [60/300], Loss: 0.1960
Epoch [90/300], Loss: 0.0686
Epoch [120/300], Loss: 0.0328
Epoch [150/300], Loss: 0.0198
Epoch [180/300], Loss: 0.0134
Epoch [210/300], Loss: 0.0098
Epoch [240/300], Loss: 0.0076
Epoch [270/300], Loss: 0.0061
Epoch [300/300], Loss: 0.0051


### GRU 구현

In [6]:
class GRU(nn.Module): # GRU 클래스 선언
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU, self).__init__() # nn.Module의 초기화 함수 상속
        self.D = input_size # 입력 차원의 크기 지정
        self.M = hidden_size # hidden state의 크기 지정
        self.K = output_size # 출력 차원의 크기 지정
        self.gru = nn.GRU( # GRU 모듈 생성
            input_size=self.D, # 입력 차원의 크기
            hidden_size=self.M, # hidden state의 크기
            batch_first=True # 첫 번째 차원이 batch size임을 명시
        )
        self.fc = nn.Linear(self.M, self.K)

    def forward(self, X): # 순전파 함수 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(device) # 초기 hidden state를 0으로 초기화

        out, _ = self.gru(X, h0) # GRU에 입력을 전달하고 출력과 hidden state를 받음

        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

### GRU 학습

In [9]:
model = GRU(input_size=2, hidden_size=20, output_size=2).to(device) # 모델 생성
criterion = nn.CrossEntropyLoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

for epoch in range(300):
    model.zero_grad() # 기울기 초기화
    outputs = model(inputs) # 모델 출력 계산
    loss = criterion(outputs, torch.tensor([0]).to(device)) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 30 == 0: 
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 300, loss.item()))

Epoch [30/300], Loss: 0.1834
Epoch [60/300], Loss: 0.0659
Epoch [90/300], Loss: 0.0347
Epoch [120/300], Loss: 0.0225
Epoch [150/300], Loss: 0.0161
Epoch [180/300], Loss: 0.0123
Epoch [210/300], Loss: 0.0098
Epoch [240/300], Loss: 0.0080
Epoch [270/300], Loss: 0.0067
Epoch [300/300], Loss: 0.0057


### One-to-One RNN
이 모델은 각 입력에 대해 하나의 출력을 생성하는 기본적인 RNN 구조로 예를 들어 주어진 숫자의 제곱을 예측하는 문제를 해결할 수 있다. 

In [11]:
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성, 1~4 사이의 정수 1000개를 랜덤하게 선택
Y = np.square(X) # 출력 데이터 생성, 입력에 대한 제곱을 계산

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).squeeze(-1).to(device) # 출력 데이터를 텐서로 변환

model = SimpleRNN(input_size=1, hidden_size=40, output_size=1).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

for epoch in range(4000):
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델 출력 계산
    loss = criterion(outputs, Y) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 100 == 0: 
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 4000, loss.item()))

# Infenrence
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print(f"Input: 2.0, Output: {model(X_test).item()}, 정답: {np.square(2.0)}") # 예측값 출력

Epoch [100/4000], Loss: 43.8084
Epoch [200/4000], Loss: 22.8585
Epoch [300/4000], Loss: 17.2761
Epoch [400/4000], Loss: 12.4721
Epoch [500/4000], Loss: 7.7927
Epoch [600/4000], Loss: 4.5822
Epoch [700/4000], Loss: 3.0815
Epoch [800/4000], Loss: 2.5274
Epoch [900/4000], Loss: 2.2812
Epoch [1000/4000], Loss: 2.1124
Epoch [1100/4000], Loss: 1.9652
Epoch [1200/4000], Loss: 1.8250
Epoch [1300/4000], Loss: 1.6867
Epoch [1400/4000], Loss: 1.5481
Epoch [1500/4000], Loss: 1.4085
Epoch [1600/4000], Loss: 1.2680
Epoch [1700/4000], Loss: 1.1277
Epoch [1800/4000], Loss: 0.9892
Epoch [1900/4000], Loss: 0.8546
Epoch [2000/4000], Loss: 0.7261
Epoch [2100/4000], Loss: 0.6059
Epoch [2200/4000], Loss: 0.4960
Epoch [2300/4000], Loss: 0.3979
Epoch [2400/4000], Loss: 0.3124
Epoch [2500/4000], Loss: 0.2399
Epoch [2600/4000], Loss: 0.1800
Epoch [2700/4000], Loss: 0.1318
Epoch [2800/4000], Loss: 0.0941
Epoch [2900/4000], Loss: 0.0655
Epoch [3000/4000], Loss: 0.0444
Epoch [3100/4000], Loss: 0.0293
Epoch [3200/4

### One-to-Many
One-to-Many 구조의 모델은 하나의 입력에 대해 여러 개의 출력을 생성하는 구조이다. 예를 들어 주어진 수자의 배수를 예측하는 문제를 해결할 수 있다.

In [15]:
# 입력의 배수 10개를 타겟으로 설정
# 아래의 One-to-Many는 하나의 숫자 입력 데이터를 받고 입력 데이터의 배수 10개를 출력하는 RNN 모델

class SimpleRNNOne2Many(nn.Module): # One-to-Many RNN 클래스 선언
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNNOne2Many, self).__init__()
        self.D = input_size
        self.M = hidden_size
        self.K = output_size
        self.rnn = nn.RNN(
            input_size=self.D,
            hidden_size=self.M,
            nonlinearity='tanh',
            batch_first=True
        )
        self.fc = nn.Linear(self.M, self.K) 
    
    def forward(self, X): # 순전파 함수 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(device) # 초기 hidden state를 0으로 초기화
        out, _ = self.rnn(X, h0) # RNN에 입력을 전달하고 출력과 hidden state를 받음
        out = self.fc(out) # 선형 변환을 수행
        return out.view(-1, 10)
    
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성
Y = np.array([[i*j for i in range(1, 11)] for j in X.squeeze()]) # 출력 데이터 생성

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 출력 데이터를 텐서로 변환

model = SimpleRNNOne2Many(input_size=1, hidden_size=40, output_size=10).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

for epoch in range(4000):
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델 출력 계산
    loss = criterion(outputs, Y) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 100 == 0: 
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: 2.0")
output = [round(num, 1) for num in model(X_test).squeeze().tolist()] # 테스트 데이터에 대한 예측값 계산
answer = list(range(2, 21, 2)) # 정답 리스트 생성

for o, a in zip(output, answer): # 예측값과 정답을 비교하여 출력
    print(f"Output: {o}, 정답: {a}")

Epoch [100/4000], Loss: 224.2033
Epoch [200/4000], Loss: 130.7445
Epoch [300/4000], Loss: 79.7772
Epoch [400/4000], Loss: 55.0492
Epoch [500/4000], Loss: 42.4101
Epoch [600/4000], Loss: 34.5626
Epoch [700/4000], Loss: 28.0176
Epoch [800/4000], Loss: 21.6280
Epoch [900/4000], Loss: 15.8146
Epoch [1000/4000], Loss: 11.1943
Epoch [1100/4000], Loss: 7.9706
Epoch [1200/4000], Loss: 5.9303
Epoch [1300/4000], Loss: 4.6914
Epoch [1400/4000], Loss: 3.9157
Epoch [1500/4000], Loss: 3.3828
Epoch [1600/4000], Loss: 2.9746
Epoch [1700/4000], Loss: 2.6361
Epoch [1800/4000], Loss: 2.3415
Epoch [1900/4000], Loss: 2.0778
Epoch [2000/4000], Loss: 1.8374
Epoch [2100/4000], Loss: 1.6162
Epoch [2200/4000], Loss: 1.4117
Epoch [2300/4000], Loss: 1.2228
Epoch [2400/4000], Loss: 1.0491
Epoch [2500/4000], Loss: 0.8906
Epoch [2600/4000], Loss: 0.7475
Epoch [2700/4000], Loss: 0.6202
Epoch [2800/4000], Loss: 0.5085
Epoch [2900/4000], Loss: 0.4121
Epoch [3000/4000], Loss: 0.3303
Epoch [3100/4000], Loss: 0.2622
Epoch

### Many-to-One
Many-to-One 구조의 모델은 여러개의 입력에 대해 하나의 출력을 생성한다. 예를 들어 주어진 숫자 리스트의 합을 예측하는 문제를 해결할 수 있다.

In [5]:
# 아래의 Many-to-One은 여러 개의 숫자 입력 데이터를 받고 입력된 데이터의 총 합을 예측하는 RNN 모델

class ManyToOneRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ManyToOneRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 모듈 생성
        self.fc = nn.Linear(hidden_size, output_size) # 출력을 위한 선형 변환 정의

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # 초기 hidden state를 0으로 초기화
        out, _ = self.rnn(x, h0) # RNN에 입력을 전달하고 출력과 hidden state를 받음
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out
    
X = np.random.randint(1, 15, size=(50000, 6, 1)) # 입력 데이터 생성
Y = np.array([np.sum(x) for x in X]) # 출력 데이터 생성

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 출력 데이터를 텐서로 변환

model = ManyToOneRNN(input_size=1, hidden_size=50, output_size=1).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

for epoch in range(4000):
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델 출력 계산
    loss = criterion(outputs.squeeze(), Y) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 100 == 0: 
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0], [4.0], [6.0], [8.0], [10.0], [11.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {X_test.squeeze().tolist()}")
output = round(model(X_test).item(), 1)
answer = round(sum(X_test.squeeze().tolist()), 1)

print(f"Output: {output}, 정답: {answer}")

Epoch [100/4000], Loss: 1517.0813
Epoch [200/4000], Loss: 1144.5833
Epoch [300/4000], Loss: 866.0313
Epoch [400/4000], Loss: 652.1705
Epoch [500/4000], Loss: 489.3702
Epoch [600/4000], Loss: 367.6650
Epoch [700/4000], Loss: 278.7727
Epoch [800/4000], Loss: 215.5821
Epoch [900/4000], Loss: 172.0082
Epoch [1000/4000], Loss: 142.9507
Epoch [1100/4000], Loss: 124.2650
Epoch [1200/4000], Loss: 112.7084
Epoch [1300/4000], Loss: 105.8512
Epoch [1400/4000], Loss: 101.9564
Epoch [1500/4000], Loss: 99.8430
Epoch [1600/4000], Loss: 98.7491
Epoch [1700/4000], Loss: 98.2091
Epoch [1800/4000], Loss: 97.9534
Epoch [1900/4000], Loss: 97.8331
Epoch [2000/4000], Loss: 97.7624
Epoch [2100/4000], Loss: 93.7729
Epoch [2200/4000], Loss: 57.0565
Epoch [2300/4000], Loss: 34.1876
Epoch [2400/4000], Loss: 24.8786
Epoch [2500/4000], Loss: 19.2039
Epoch [2600/4000], Loss: 15.3059
Epoch [2700/4000], Loss: 12.4426
Epoch [2800/4000], Loss: 10.2776
Epoch [2900/4000], Loss: 8.5976
Epoch [3000/4000], Loss: 7.2794
Epoch

### Many-to-Many RNN
Many-to-Many 구조의 모델은 여러 개의 입력에 대해 여러 개의 출력을 생성한다. 예를 들어 주어진 숫자 리스트의 누적 합을 예측하는 문제를 해결할 수 있다.

In [8]:
# 입력 데이터 리스트의 누적 합 리스트가 정답
# 아래의 Many-to-Many 구조는 여러 개의 숫자 입력 데이터를 받고 입력 데이터의 누적합을 예측하는 RNN 모델

class ManyToManyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ManyToManyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 모듈 생성
        self.fc = nn.Linear(hidden_size, output_size) # 출력을 위한 선형 변환 정의

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # 초기 hidden state를 0으로 초기화
        out, _ = self.rnn(x, h0) # RNN에 입력을 전달하고 출력과 hidden state를 받음
        out = self.fc(out) # 선형 변환을 수행

        return out
    
X = np.array([[[np.random.randint(0, 31)] for _ in range(5)] for _ in range(3000)])
Y = np.array([np.cumsum(x) for x in X]) 

X = torch.from_numpy(X.astype(np.float32)) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)) # 출력 데이터를 텐서로 변환

model = ManyToManyRNN(1, 60, 1)
criterion = nn.MSELoss() # 손실 함수 정의
optimizer = torch.optim.Adam(model.parameters()) # 옵티마이저 정의

for epoch in range(4000):
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델 출력 계산
    loss = criterion(outputs.squeeze(), Y) # 손실 계산
    loss.backward() # 역전파
    optimizer.step() # 파라미터 업데이트

    if (epoch + 1) % 100 == 0: 
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[i+10] for i in range(5)]], dtype=torch.float32) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {list(range(10, 15))}")
output = [round(num, 1) for num in model(X_test).squeeze().tolist()] # 모델의 출력 계산
answer = list(np.cumsum(range(10, 15))) # 정답 계산

for o, a in zip(output, answer): # 모델의 출력과 정답 비교
    print(f"Output: {o}, 정답: {a}")

Epoch [100/4000], Loss: 1953.6315
Epoch [200/4000], Loss: 1548.8319
Epoch [300/4000], Loss: 1250.9359
Epoch [400/4000], Loss: 1019.4675
Epoch [500/4000], Loss: 837.5214
Epoch [600/4000], Loss: 690.1598
Epoch [700/4000], Loss: 567.5901
Epoch [800/4000], Loss: 469.3835
Epoch [900/4000], Loss: 390.6965
Epoch [1000/4000], Loss: 327.0983
Epoch [1100/4000], Loss: 275.5186
Epoch [1200/4000], Loss: 233.3695
Epoch [1300/4000], Loss: 198.6822
Epoch [1400/4000], Loss: 169.9465
Epoch [1500/4000], Loss: 146.0302
Epoch [1600/4000], Loss: 126.0160
Epoch [1700/4000], Loss: 109.1813
Epoch [1800/4000], Loss: 94.9762
Epoch [1900/4000], Loss: 82.9228
Epoch [2000/4000], Loss: 72.6546
Epoch [2100/4000], Loss: 63.8635
Epoch [2200/4000], Loss: 56.3131
Epoch [2300/4000], Loss: 49.8050
Epoch [2400/4000], Loss: 44.1751
Epoch [2500/4000], Loss: 39.2975
Epoch [2600/4000], Loss: 35.0264
Epoch [2700/4000], Loss: 31.3073
Epoch [2800/4000], Loss: 28.1150
Epoch [2900/4000], Loss: 25.1898
Epoch [3000/4000], Loss: 22.670